# Análise das Ocorrências Aeronáuticas na Aviação Civil Brasileira

**Data: 09-06-2022**

**Autor: Jaildo Rocha**

---

No presente notebook busca-se obter, a partir da análise e exploração de dados, uma compreensão sobre causas das [ocorrências aeronáuticas na aviação civil](https://dados.gov.br/dataset/ocorrencias-aeronauticas-da-aviacao-civil-brasileira) e obter insights de possíveis melhorias na segurança da aviação civil. Para isso, este estudo foi dividido em etapas:

* Pré processamento (importar pacotes e dados);
* Verificação  e limpeza dos dados;
* Análise exploratórias dos dados;
* Conclusões; e
* Recomendações para o prosseguimento do trabalho. 

Cabe realizar duas observações importantes para este estudo. O primeiro é que este estudo tem foco, principalmente, em aviões e helicópteros. Tal premissa decorre do entendimento que outros tipos de aeronaves (Trike, balão, ultraleve, planador, jidroavião e anfíbio) tem aspectos regulatórios que diferem bastante dos aviões e helicópteros. O segundo é que, pontualmente, serão utilizadas outras bases de dados, estudos ou informações para contextualizar este estudo.

## Pré processamento

Primeiramente, realiza-se a importação das bibliotecas e uma configuração básica para as figuras.

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# set plot parameters
custom_params = {'axes.spines.right': False, 
                 'axes.spines.top': False,
                 'figure.figsize':(16,8)}
sns.set_theme(style='ticks', rc = custom_params)

Em seguida os dados das ocorrências são importados.

In [20]:
# tipos de valores não disponíveis
list_na_values = ['*', '**', '***', '****', '*****',
            'NULL', 'NÃO INFORMADO', '###', '###!', '####',]

# read ocorrencia
ocorrencia_df = pd.read_csv('../raw_data/ocorrencia.csv',
                            sep = ';', encoding='utf8', 
                            na_values = list_na_values)

# read tipo ocorrencia
tipo_occ_df = pd.read_csv('../raw_data/ocorrencia_tipo.csv',
                          sep = ';', encoding = 'utf8', 
                          na_values = list_na_values)

# read aeronave
aeronave_df = pd.read_csv('../raw_data/aeronave.csv',
                          sep = ';', encoding = 'utf8', 
                          na_values = list_na_values)

# read fator contribuinte
fator_cont_df = pd.read_csv('../raw_data/fator_contribuinte.csv',
                            sep = ';', encoding = 'utf8', 
                            na_values = list_na_values)

# read recomendacao
recomendacao_df = pd.read_csv('../raw_data/recomendacao.csv',
                              sep = ';', encoding = 'utf8', 
                              na_values = list_na_values)

Em seguida se faz uma breve verificação dos dados importados.

### Ocorrências

In [21]:
ocorrencia_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5167 entries, 0 to 5166
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   codigo_ocorrencia               5167 non-null   int64 
 1   codigo_ocorrencia1              5167 non-null   int64 
 2   codigo_ocorrencia2              5167 non-null   int64 
 3   codigo_ocorrencia3              5167 non-null   int64 
 4   codigo_ocorrencia4              5167 non-null   int64 
 5   ocorrencia_classificacao        5167 non-null   object
 6   ocorrencia_latitude             3673 non-null   object
 7   ocorrencia_longitude            3672 non-null   object
 8   ocorrencia_cidade               5167 non-null   object
 9   ocorrencia_uf                   5165 non-null   object
 10  ocorrencia_pais                 5167 non-null   object
 11  ocorrencia_aerodromo            3262 non-null   object
 12  ocorrencia_dia                  5167 non-null   

In [26]:
ocorrencia_df.nunique()

codigo_ocorrencia                 5167
codigo_ocorrencia1                5167
codigo_ocorrencia2                5167
codigo_ocorrencia3                5167
codigo_ocorrencia4                5167
ocorrencia_classificacao             3
ocorrencia_latitude               2586
ocorrencia_longitude              2589
ocorrencia_cidade                 1053
ocorrencia_uf                       27
ocorrencia_pais                      1
ocorrencia_aerodromo               512
ocorrencia_dia                    2684
ocorrencia_hora                    923
investigacao_aeronave_liberada       2
investigacao_status                  2
divulgacao_relatorio_numero       1718
divulgacao_relatorio_publicado       2
divulgacao_dia_publicacao          179
total_recomendacoes                 13
total_aeronaves_envolvidas           3
ocorrencia_saida_pista               2
dtype: int64

Como pode ser observado, há apenas uma ocorrência para cada linha. Aparentemente, os dados apresentam uma boa qualidade.

### Tipo de ocorrência

In [27]:
tipo_occ_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5347 entries, 0 to 5346
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   codigo_ocorrencia1         5347 non-null   int64 
 1   ocorrencia_tipo            5346 non-null   object
 2   ocorrencia_tipo_categoria  5346 non-null   object
 3   taxonomia_tipo_icao        5346 non-null   object
dtypes: int64(1), object(3)
memory usage: 167.2+ KB


In [28]:
tipo_occ_df.nunique()

codigo_ocorrencia1           5167
ocorrencia_tipo                80
ocorrencia_tipo_categoria      80
taxonomia_tipo_icao            32
dtype: int64

No caso do tipo de ocorrência, um evento pode apresentar mais de um tipo ocorrência. Além disso, os dados aparentemente estão bem classificados.

### Aeronave

In [23]:
aeronave_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5235 entries, 0 to 5234
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   codigo_ocorrencia2           5235 non-null   int64  
 1   aeronave_matricula           5221 non-null   object 
 2   aeronave_operador_categoria  2287 non-null   object 
 3   aeronave_tipo_veiculo        5080 non-null   object 
 4   aeronave_fabricante          4887 non-null   object 
 5   aeronave_modelo              5065 non-null   object 
 6   aeronave_tipo_icao           4970 non-null   object 
 7   aeronave_motor_tipo          5003 non-null   object 
 8   aeronave_motor_quantidade    5143 non-null   object 
 9   aeronave_pmd                 5235 non-null   int64  
 10  aeronave_pmd_categoria       5235 non-null   int64  
 11  aeronave_assentos            5041 non-null   float64
 12  aeronave_ano_fabricacao      5035 non-null   float64
 13  aeronave_pais_fabr

In [29]:
aeronave_df.nunique()

codigo_ocorrencia2             5167
aeronave_matricula             3901
aeronave_operador_categoria      10
aeronave_tipo_veiculo             9
aeronave_fabricante             233
aeronave_modelo                 737
aeronave_tipo_icao              229
aeronave_motor_tipo               5
aeronave_motor_quantidade         5
aeronave_pmd                    459
aeronave_pmd_categoria          459
aeronave_assentos                86
aeronave_ano_fabricacao          80
aeronave_pais_fabricante         23
aeronave_pais_registro           23
aeronave_registro_categoria       9
aeronave_registro_segmento       12
aeronave_voo_origem             677
aeronave_voo_destino            674
aeronave_fase_operacao           31
aeronave_tipo_operacao            9
aeronave_nivel_dano               4
aeronave_fatalidades_total       10
dtype: int64

No caso do dataframe das aeronaves, são observados uma maior quantidade de problemas, uma vez que, algumas aeronaves não apresentam matricula, alguns veículos não são tipificados, a classificação quanto ao peso máximo de decolagem (PMD) parece não existir.

### Fator de contribuição para ocorrência

In [31]:
fator_cont_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3464 entries, 0 to 3463
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   codigo_ocorrencia3   3464 non-null   int64 
 1   fator_nome           3464 non-null   object
 2   fator_aspecto        3445 non-null   object
 3   fator_condicionante  3110 non-null   object
 4   fator_area           3445 non-null   object
dtypes: int64(1), object(4)
memory usage: 135.4+ KB
